<a href="https://colab.research.google.com/github/abhk-11/text_summarization/blob/main/bert_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spacy

NotImplementedError: ignored

In [ ]:
from torch.optim.lr_scheduler import _LRScheduler

In [ ]:
import pandas as pd

In [ ]:
pip install transformers

NotImplementedError: ignored

In [ ]:
pip install sentence-transformers

In [ ]:
import spacy

In [ ]:
nlp=spacy.load("en_core_web_sm")

In [ ]:
import json

In [ ]:
data=json.load(open('/content/json_1.json'))

In [ ]:
data1=json.load(open('/content/json_2.json'))

In [ ]:
data2=json.load(open('/content/json_3.json'))

In [ ]:
lst=[data,data1,data2]

In [ ]:
input=[]
output=[]

In [ ]:
for dat in lst:
  for key , val in dat.items():
    input.append(key)
    output.append(val)

In [ ]:
str1=' '.join(input)
str2=' '.join(output)

In [ ]:
str1=str1.replace("we’ll",'we will')
str1=str1.replace("We’ll",'We will')

In [ ]:
my_doc=nlp(str1)

In [ ]:
doc2=nlp(' '.join(output))

In [ ]:
in_sent=[tok for tok in my_doc.sents]

In [ ]:
out_sent=[tok for tok in doc2.sents]

In [ ]:
for i in range(7,15):
 print(in_sent[i].similarity(out_sent[2]))

In [ ]:
in_lb=[]
out_lb=[]

In [ ]:
len(in_sent)

In [ ]:
len(out_sent)

In [ ]:
for sent in in_sent:
  in_lb.append([sent,0])

In [ ]:
for sent in out_sent:
  out_lb.append([sent,0])

In [ ]:
j=1
k=int(len(in_sent)/len(out_sent))
while (int(k*(j+1)+1)<len(in_sent)-1 and j<len(out_sent)-1):
  j=j+1
  t=0
  index=0
  for i in range(int(k*(j)),int(k*(j+1))):
    if in_sent[i].similarity(out_sent[j])>t:
      t=in_sent[i].similarity(out_sent[j])
      index=i
  in_lb[index][1]=1

In [ ]:
for i in range(0,4):
    if in_sent[i].similarity(out_sent[j])>t:
      t=in_sent[i].similarity(out_sent[0])
      index=i
in_lb[index][1]=1

In [ ]:
for i in range(len(in_sent)-5,len(in_sent)-1):
    if in_sent[i].similarity(out_sent[j])>t:
      t=in_sent[i].similarity(out_sent[142])
      index=i
in_lb[index][1]=1

In [ ]:
j=0
for i in in_lb:
  if i[1]==1:
    j=j+1
    print(i[0])

In [ ]:
j

In [ ]:
df=pd.DataFrame(in_lb,columns=['input','label'])

In [ ]:
df.head()

In [ ]:
(in_lb[5][0], in_lb[5][1])

In [ ]:
rw_input=[]

In [ ]:
for i in range(0,len(in_lb)):
  rw_input.append((in_lb[i][0].text,in_lb[i][1]))

In [ ]:
z=0
for tx in rw_input:
  print(tx)
  z=z+1
  if(z==10):
    break

In [ ]:
import torch
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_seq_length = 128

input_sentences = []
labels = []
attention_masks = []

for sent in rw_input:
    sentence, label = sent[0], sent[1]

    tokens = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=max_seq_length, pad_to_max_length=True, return_attention_mask=True)

    input_sentences.append(tokens['input_ids'])
    attention_masks.append(tokens['attention_mask'])

    labels.append(torch.tensor(label))

input_ids = torch.tensor(input_sentences)
attention_masks = torch.tensor(attention_masks)
labels = torch.stack(labels)


In [ ]:
def get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, last_epoch=-1):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps)))

    return LambdaLR(optimizer, lr_lambda, last_epoch)


In [ ]:
class LambdaLR(_LRScheduler):
    def __init__(self, optimizer, lr_lambda, last_epoch=-1):
        self.lr_lambda = lr_lambda
        super(LambdaLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.lr_lambda(self.last_epoch) * base_lr for base_lr in self.base_lrs]


In [ ]:
import torch
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)

batch_size = 32

dataset = TensorDataset(input_ids, attention_masks, labels)

sampler = RandomSampler(dataset)

dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)

num_epochs = 3

learning_rate = 2e-5

optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataloader)*num_epochs)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_loss / len(dataloader)
    print("Average training loss: {0:.2f}".format(avg_train_loss))

torch.save(model.state_dict(), "bert_model.bin")
